In [0]:
%pip install spacy

In [0]:
# Import necessary modules
import sys
sys.path.append('./')
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
import importlib

# Import UDF functions from modules
import udf_further_term
import udf_total_term

# Reload the modules to ensure the latest version
udf_further_term = importlib.reload(udf_further_term)
udf_total_term = importlib.reload(udf_total_term)

# Register UDF functions
spark.udf.register("extract_total_term", udf_further_term.extract_total_term, IntegerType())
spark.udf.register("parse_and_compute", udf_total_term.parse_and_compute_udf)


## Generate Predictions from the lease model

In [0]:
%sql
-- Run process_table_with_spacy_model first
Create or Replace Temp View EscalationRate as
with default_escaltion as(
select
ma.id,
ma.rentincreasedescription,
ma.specialconditions,
CASE WHEN ec.extracted_entities IS NULL THEN REGEXP_EXTRACT(ma.specialconditions, '(\\b\\d+\\.?\\d*%)', 0)
ELSE ec.extracted_entities end as extracted_entities
FROM `arealytics-databricks_unity_catalog`.arealyticstrusted.reiv_lease_main as ma

left join reiv_escalation_delta ec on
ec.id = ma.id
)

select
ma.id,
dc.extracted_entities,
ROUND(REGEXP_EXTRACT(dc.extracted_entities, '(\\d+\\.?\\d*)%?', 1), 0) as two2,
CASE WHEN ROUND(REGEXP_EXTRACT(dc.extracted_entities, '(\\d+\\.?\\d*)%?', 1), 0)  BETWEEN 1 AND 9 THEN ROUND(REGEXP_EXTRACT(dc.extracted_entities, '(\\d+\\.?\\d*)%?', 1), 0) 
        ELSE NULL
END AS EscalationRate 
FROM `arealytics-databricks_unity_catalog`.arealyticstrusted.reiv_lease_main as ma

left join default_escaltion as dc on
dc.id = ma.id


## Create Suite temp table

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW reiv_suite_cleanup as
select 
ma.id
,CASE
     WHEN unitnumber IS NULL OR unitnumber = '' THEN Geo.unit
     wHEN unitnumber IS NULL OR unitnumber = '' and Geo.unit is null then
        CASE
              -- Extracts a number at the beginning of the address
              WHEN streetaddress REGEXP '^\\d+' THEN REGEXP_SUBSTR(streetaddress, '^\\d+')
              -- Extracts the first number sequence before any slash, if present
              WHEN streetaddress REGEXP '\\b\\d+(?=/)' THEN REGEXP_SUBSTR(streetaddress, '\\b\\d+(?=/)')
              -- For other address types like Suite, Warehouse, etc.
              WHEN streetaddress REGEXP '\\bSuite\\s*\\d+' THEN REGEXP_SUBSTR(streetaddress, '\\bSuite\\s*\\d+')
              WHEN streetaddress REGEXP '\\bWarehouse\\s*\\d+' THEN REGEXP_SUBSTR(streetaddress, '\\bWarehouse\\s*\\d+')
              WHEN streetaddress REGEXP '\\bBuilding\\s*\\d+' THEN REGEXP_SUBSTR(streetaddress, '\\bBuilding\\s*\\d+')
              WHEN streetaddress REGEXP '\\bLevel\\s*\\d+' THEN REGEXP_SUBSTR(streetaddress, '\\bLevel\\s*\\d+')
              WHEN streetaddress REGEXP '\\bLot\\s*\\d+' THEN REGEXP_SUBSTR(streetaddress, '\\bLot\\s*\\d+')
              WHEN streetaddress REGEXP '\\bNo\\.\\s*\\d+' THEN REGEXP_SUBSTR(streetaddress, '\\bNo\\.\\s*\\d+')
               -- Extract patterns like 'Unit 2', 'Shop A', 'Suite 1B', etc.
              WHEN lower(streetaddress) REGEXP '\\b(unit|shop|suite|suites|rear|shed)\\s*\\d+[A-Za-z0-9-]*' THEN REGEXP_SUBSTR(lower(streetaddress), '\\b(unit|shop|suite|suites|rear|shed)\\s*\\d+[A-Za-z0-9-]*')
              WHEN streetaddress REGEXP '^\\d+[^\\d\\s]*\\b(?!\\s*\\d{4}\\b)' THEN REGEXP_SUBSTR(streetaddress, '^\\d+[^\\d\\s]*')
              ELSE NULL
          END
      ELSE
          unitnumber
  END AS Suite
from `arealytics-databricks_unity_catalog`.arealyticstrusted.reiv_lease_main as ma
left join `arealytics-databricks_unity_catalog`.arealyticstrusted.reiv_geocoded_data geo
on ma.id = geo.id

**Creating reiv_term_dates View**

The temporary view `reiv_term_dates` which organizes and standardizes the term dates from the `reinsw_main` table wheb there is no date when calulate a defualt date.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW reiv_term_dates as
with cte as (
select 
ma.id,
ma.created_at,
CAST(commencementdate AS DATE) AS CommencingDate,
CASE
    WHEN CAST(commencementdate AS DATE) = NULL OR extract_total_term(ma.termpremises) = 0 THEN NULL
    WHEN  extract_total_term(ma.termpremises) != 0 OR CAST(commencementdate AS DATE) != NULL THEN CAST(DATEADD(MONTH, extract_total_term(ma.termpremises), CAST(commencementdate AS DATE)) AS DATE)
    ELSE NULL
END AS ExpiryDate
from `arealytics-databricks_unity_catalog`.arealyticstrusted.reiv_lease_main as ma
),

CommencingDate_cte as (
Select  
id,
ExpiryDate,
CommencingDate as CommencingDate_org,
created_at,
date_format(created_at, 'yyyy-MM-dd') as created,
CASE
        -- Calculate Commencement Date if Expiry is NOT NULL and Commencement is NULL
        WHEN ExpiryDate IS NOT NULL AND CommencingDate IS NULL THEN
            date_format(DATE_ADD(ExpiryDate, -729), 'yyyy-MM-dd')
        -- Calculate Commencement Date if Expiry is NULL and Commencement is NULL
        WHEN ExpiryDate IS NULL AND CommencingDate IS NULL THEN
           date_format(DATE_ADD(DATE_ADD(Created, 30), 1), 'yyyy-MM-dd')
        -- Keep existing Commencement Date if it's not NULL
        WHEN CommencingDate IS NULL OR (ExpiryDate IS NOT NULL AND ExpiryDate <= CommencingDate) THEN
            date_format(DATE_ADD(DATE_ADD(Created, 30), 1), 'yyyy-MM-dd')
        ELSE
          date_format(CommencingDate, 'yyyy-MM-dd')
    END AS CommencingDate
from cte
),

ExpiryDate_cte as (
Select  
id,
ExpiryDate as Expiry_Org,
CommencingDate_org,
created,
    CASE
        -- Calculate Expiry Date if both Expiry and Commencement are NULL
        WHEN ExpiryDate IS NULL AND CommencingDate IS NULL THEN
            DATE_FORMAT(DATE_ADD(Created, 30 + 730), 'yyyy-MM-dd')
        -- Calculate Expiry Date if Commencement is NOT NULL and Expiry is NULL
        WHEN CommencingDate IS NOT NULL AND ExpiryDate IS NULL THEN
            DATE_FORMAT(DATE_ADD(CommencingDate, 730), 'yyyy-MM-dd')
        ELSE
            DATE_FORMAT(ExpiryDate, 'yyyy-MM-dd')
    END AS ExpiryDate,
CommencingDate
from CommencingDate_cte
)
select 
*,
CASE 
        WHEN CommencingDate IS NOT NULL AND CommencingDate_org is null then 1
        WHEN DATE_FORMAT(CommencingDate, 'yyyy-MM-dd') != DATE_FORMAT(CommencingDate_org, 'yyyy-MM-dd') then 1
        ELSE 0
END AS Commencing_Estimated_Flag,
Case
        WHEN ExpiryDate IS NOT NULL AND Expiry_Org is null then 1
        WHEN DATE_FORMAT(ExpiryDate, 'yyyy-MM-dd') != DATE_FORMAT(Expiry_Org, 'yyyy-MM-dd') then 1
        ELSE 0
    END AS ExpiryDate_Estimated_Flag
from ExpiryDate_cte

Curate Lease Main

In [0]:
%sql
-- Set the load date variable
-- SET loadDate = CURRENT_DATE();

CREATE OR REPLACE TABLE `arealytics-databricks_unity_catalog`.arealyticscurated.reiv_lease_main
AS

SELECT
    distinct
    pd.PropertyID AS PropertyID,
    ma.created_at,
    CASE
        WHEN GPid.cleaned_unit IS NOT NULL AND GPid.street_number IS NOT NULL THEN TRIM(CONCAT(GPid.cleaned_unit, '/', GPid.street_number, ' ', COALESCE(GPid.Cleaned_Street_Name, '')))
        WHEN GPid.cleaned_unit IS NOT NULL THEN TRIM(CONCAT(GPid.cleaned_unit, ' ', COALESCE(GPid.Cleaned_Street_Name, '')))
        WHEN GPid.street_number IS NOT NULL THEN TRIM(CONCAT(GPid.street_number, ' ', COALESCE(GPid.Cleaned_Street_Name, '')))
        ELSE NULL
    END as PropertyName,
    CASE
        WHEN GPid.street_number IS NOT NULL THEN TRIM(CONCAT(GPid.street_number, ' ', COALESCE(GPid.Cleaned_Street_Name, '')))
        ELSE NULL
    END AS Address,
    GPid.suburb AS City, -- geo suburb
    cast(null as string) AS State,
    GPid.postal_code AS Zip, -- geo post code
    initcap(ma.tenantname ) AS AltTenantName, --Get tenant id
    CASE
        WHEN LOWER(streetaddress) RLIKE 'ground floor' THEN 'GF'
        WHEN LOWER(streetaddress) RLIKE 'level\\s*\\d+' THEN REGEXP_EXTRACT(LOWER(streetaddress), 'level\\s*(\\d+)', 1)
        ELSE NULL
    END AS Level,
    -- SuiteNumber logic
    CASE    
        WHEN suite.suite IS NOT NULL THEN REGEXP_SUBSTR(suite.suite, '(\\b\\d+)')
        ELSE NULL
    END AS Suite,
    cast(null as string) AS LeasedSqm, -- Field not in original select
    DATE_FORMAT(td.CommencingDate, 'dd/MM/yyyy') AS CommencingDate,
    -- ExpirationDate logic
    DATE_FORMAT(td.ExpiryDate, 'dd/MM/yyyy') AS ExpiryDate,
    CASE 
        WHEN lower(preferredrentoption) = Lower('Annum') THEN ROUND(commencingrentamount, 0)
        WHEN lower(preferredrentoption) = Lower('calendarMonth') THEN ROUND(commencingrentamount * 12, 0)
    ELSE NULL
    END AS FaceRentAnnual,
    -- LeaseRateTypeID logic
    CASE 
        WHEN outgoingsapportionmentdescription REGEXP '[0-9]+%' OR outgoingsapportionmentdescription LIKE '%outgoings%' THEN 10
        WHEN outgoingsapportionmentdescription = 'Nil' OR outgoingsapportionmentdescription LIKE 'Nil%' THEN 4
        ELSE NULL
    END AS LeaseRateType,
    cast(null as string) AS OutgoingsPerSqm,
    -- EscalationRate logic  -- REGEXP_EXTRACT(ec.extracted_entity, '(\\d+\\.?\\d*)%?', 1)
    ec.EscalationRate AS EscalationRate,
     -- Total term logic   
    CASE 
        WHEN td.CommencingDate IS NOT NULL AND td.ExpiryDate IS NOT NULL THEN
            ROUND(MONTHS_BETWEEN(td.ExpiryDate, td.CommencingDate), 0)
        WHEN extract_total_term(ma.termpremises) = 0 THEN
            NULL 
        ELSE 
            extract_total_term(ma.termpremises) 
    END AS TermMonths,
    CASE 
        WHEN parse_and_compute(furthertermsdescription).result <= 0 THEN NULL 
    ELSE parse_and_compute(furthertermsdescription).result
    END AS RenewalOptionsNoOfMonths,
    CASE 
        WHEN ma.created_at != 'nan' THEN date_format(cast(ma.created_at as date), 'dd/MM/yyyy')
        ELSE NULL  
    END AS RecordedDate,
   CASE
        WHEN lease_type = 'lease-renewal' THEN 2
        WHEN lease_type = 'lease-schedule' THEN 1
        ELSE NULL
    END AS LeaseSublease,
    -- LeaseTransType logic
    CASE
        WHEN lease_type = 'lease-renewal' THEN 2
        WHEN lease_type = 'lease-schedule' THEN 1
        ELSE NULL
    END AS LeaseTransType,
    -- TransactionComments logic
    CASE 
        WHEN lease_type = 'lease-schedule' THEN 
            CONCAT('Renewal Option: ', FurtherTermsDescription,' ', 
                    'Outgoings Description: ', outgoingsapportionmentdescription,' ', 
                    'Excluded Outgoings: ', excludedoutgoingsdescription)
        WHEN lease_type = 'lease-renewal' THEN 
            CONCAT('Special Conditions: ', specialconditions)
        ELSE 
        NULL
    END AS TransactionComments,
    CASE 
        WHEN fixturesfittingsdescription = 'nan' and lower(fixturesfittingsdescription) = 'nil' THEN NULL
        WHEN fixturesfittingsdescription != 'nan' THEN fixturesfittingsdescription
        ELSE NULL
    END AS SuiteComments,
    CASE 
        WHEN leasedate != 'nan' THEN date_format(cast(leasedate as date), 'dd/MM/yyyy')
        ELSE NULL   
    END AS ExecutionDate,
    CASE 
        WHEN ma.permittedusetype IN ('Office, Retail', 'Unclassified') THEN 
            CASE 
                WHEN pd.PropertyUse = 'Office' THEN 9
                WHEN pd.PropertyUse = 'Industrial' THEN 8
                WHEN pd.PropertyUse = 'Retail' OR pd.PropertyUse IN ('Land', 'Multi-Family Housing') THEN 7
                ELSE 1 
            END
        WHEN ma.permittedusetype = 'Office' THEN 9
        WHEN ma.permittedusetype = 'Industrial' THEN 8
        WHEN ma.permittedusetype = 'Retail' THEN 7
        ELSE 1 -- or some other default ID for other cases
    END AS SpaceGeneralUse,
    -- LeaseStatusID logic
    CASE 
        WHEN td.ExpiryDate < CURRENT_DATE THEN 1  -- Historic
        WHEN td.ExpiryDate >= CURRENT_DATE THEN 2  -- Active
        ELSE NULL 
    END AS LeaseStatus,
    cast(null as string) AS ConfirmationNotes, -- Field not in original select
    cast(null as string) AS SignDate, -- Field not in original select
    cast(null as string) AS BankGuaranteeAmount, -- Field not in original select
    -- GSTIncluded logic
    CASE 
        WHEN commencinggstoption = 'excludeGST' THEN 0
        WHEN commencinggstoption = 'includeGST' THEN 1
        ELSE NULL
    END AS GSTIncluded,
    cast(null as string) AS IsCondo, -- Field not in original select
    -- Fitout logic
    CASE 
        WHEN fixturesfittingsdescription IS NULL 
        OR lower(fixturesfittingsdescription) LIKE lower('Nil%')
        OR fixturesfittingsdescription = 'nan' 
        THEN 0
        ELSE 1
    END AS Fitout,
    4 as TransactionOriginationTypeID,
    -- IsStartingRateConfirmed logic
    CASE 
        WHEN commencingrentamount IS NOT NULL THEN 1
        ELSE 0
    END AS IsStartingRateConfirmed,
    5 as ShareLevelID,
    -- IsExpDateConfirmed logic
    CASE 
        WHEN td.ExpiryDate_Estimated_Flag = 1 then 0
        ELSE 1
    END AS IsExpDateConfirmed,
    -- EscalationType logic
    CASE     
        WHEN ma.rentincreasedescription REGEXP '\\bCPI\\b' AND ma.rentincreasedescription REGEXP '\\bplus\\b|\\+' AND ma.rentincreasedescription REGEXP '\\d+\\.?\\d*%' THEN 'CPI/Fixed/Market'  -- CPI plus fixed %
        WHEN ma.rentincreasedescription REGEXP '\\bCPI\\b' AND ma.rentincreasedescription REGEXP '\\d+\\.?\\d*%' THEN 'CPI/Fixed'  -- Greater of CPI and fixed percentage
        WHEN ma.rentincreasedescription REGEXP '\\d+\\.?\\d*%?' THEN 'Fixed' -- Fixed percent (Searches for '%')
        WHEN ma.rentincreasedescription REGEXP '\\bCPI\\b|\\bcpi\\b|\\bCPi\\b|\\bConsumer Price Index\\b|\\b(Consumer Price Index)\\b' THEN 'CPI' -- Searches for CPI when it's on its own
        WHEN ma.rentincreasedescription REGEXP '\\bmarket\\b|\\bMarket\\b|\\bMARKET\\b' THEN 'Market' -- Searches for Market when it's on its own
        ELSE NULL
    END AS EscalationType,
    -- IsTermsConfirmed logic
    CASE 
            WHEN TermMonths > 0 and (td.Commencing_Estimated_Flag = 0 and td.ExpiryDate_Estimated_Flag = 0) THEN 1
            ELSE 0
    END AS IsTermsConfirmed,
    cast(null as string) AS ConfirmedTenantID, -- Field not in original select
    ma.id AS DealingNumber,
    -- IsOccupDateConfirmed logic
    CASE 
        WHEN td.Commencing_Estimated_Flag = 1 then 0
        ELSE 1
    END AS IsOccupDateConfirmed,
    -- RentFreePeriod logic
    CASE 
        WHEN ROUND(MONTHS_BETWEEN(
                CAST(rentcommencementdate AS DATE), 
                CAST(commencementdate AS DATE)
            ), 0) <= 0 THEN NULL
        WHEN ROUND(MONTHS_BETWEEN(
                CAST(rentcommencementdate AS DATE), 
                CAST(commencementdate AS DATE)
            ), 0) > 6 THEN NULL
        ELSE ROUND(MONTHS_BETWEEN(
                CAST(rentcommencementdate AS DATE), 
                CAST(commencementdate AS DATE)
            ), 0)
    END AS RentFreePeriod,
    -- NoticePeriodMin logic
    CASE WHEN
                ROUND(MONTHS_BETWEEN(td.ExpiryDate, CAST(exerciseoptiondate AS DATE))) <= 0 THEN NULL
                WHEN ROUND(MONTHS_BETWEEN(td.ExpiryDate, CAST(exerciseoptiondate AS DATE))) > 6 THEN NULL
                ELSE ROUND(MONTHS_BETWEEN(td.ExpiryDate, CAST(exerciseoptiondate AS DATE)))
    END AS NoticePeriodMin,
    cast(null as string) AS NoticePeriodMax, -- Field not in original select
    1 as ListingTypeID,
    CAST(NULL as string) as folio,
    CAST(NULL as string) as StageID,
    CAST(NULL as string) as Status,
    CAST(NULL as string) as ExceptionReason,
    CAST(NULL as string) as CreatedDate,
    CAST(NULL as string) as CreatedBy,
    CAST(NULL as string) as ModifiedDate,
    CAST(NULL as string) as ModifiedBy,
    CAST(NULL as string) as LeaseID,
    CAST(NULL as string) as IsActive,
    CAST(NULL as string) as BatchID,
    12 as ProviderID,
    CAST(NULL as string) as HidedBy,
    CAST(NULL as string) as IsHidden,
    CAST(NULL as string) as HidedDate,
    CAST(NULL as string) as HideReasonID,
    CAST(NULL as string) as HideReasonComments,
    CAST(NULL as string) as SubHideReasonID,
    CAST(NULL as string) as MismatchedTaggedBy,
    CAST(NULL as string) as IsMismatched,
    Gpid.latitude,
    Gpid.longitude,
    CAST(NULL as string) as `Location`,
    CAST(NULL as string) as `UID`,
    CAST(NULL as string) as `Url`,
    CAST(NULL as string) as AgentName1,
    CAST(NULL as string) as AgentEmail1,
    CAST(NULL as string) as AgentName2,
    CAST(NULL as string) as AgentEmail2,
    ROW_NUMBER() OVER (
            PARTITION BY td.CommencingDate, td.ExpiryDate, ma.tenantname, pd.PropertyID
            ORDER BY ma.created_at DESC
        ) AS rn
    
FROM `arealytics-databricks_unity_catalog`.arealyticstrusted.reiv_lease_main as ma -- 30873

LEFT JOIN EscalationRate as ec on ec.id = ma.id  -- 30873

LEFT JOIN reiv_term_dates AS td --54 821
on td.id = ma.id

LEFT JOIN `arealytics-databricks_unity_catalog`.arealyticscurated.reiv_pid_link as GPid
on GPid.id = ma.id

LEFT JOIN `arealytics-databricks_unity_catalog`.arealyticstrusted.property_detail as pd
on gpid.propertyId = pd.propertyId

LEFT JOIN reiv_suite_cleanup
as Suite on Suite.id = ma.id

LEFT JOIN `arealytics-databricks_unity_catalog`.arealyticscurated.DimCalendar as c 
ON c.DateKey = CAST(DATE_FORMAT(current_date(), 'yyyyMMdd') AS INT)

where cast(ma.LoadDate as date) = '2024-06-18'

### Export data

In [0]:
%sql
-- CREATE OR REPLACE TEMP VIEW Reiv_export AS
SELECT
    PropertyID AS `PropertyID`,
    PropertyName AS `Property Name`,
    Address,
    City,
    State,
    Zip,
    AltTenantName AS `Alt Tenant Name`,
    Level,
    Suite,
    LeasedSqm AS `Leased Sqm`,
    CommencingDate AS `Commencing Date`,
    ExpiryDate AS `Expiry Date`,
    FaceRentAnnual AS `Face Rent - Annual`,
    LeaseRateType AS `Lease Rate Type`,
    OutgoingsPerSqm AS `Outgoings (per sqm)`,
    EscalationRate AS `Escalation Rate %`,
    TermMonths AS `Term (months)`,
    RenewalOptionsNoOfMonths AS `Renewal Options (No. of Months)`,
    RecordedDate AS `Recorded Date`,
    LeaseSublease AS `Lease/Sublease`,
    LeaseTransType AS `Lease Trans Type`,
    TransactionComments AS `Transaction Comments`,
    SuiteComments AS `Suite Comments`,
    ExecutionDate AS `Execution Date`,
    SpaceGeneralUse AS `Space General Use`,
    LeaseStatus AS `Lease Status`,
    ConfirmationNotes,
    SignDate AS `SignDate`,
    BankGuaranteeAmount AS `BankGuaranteeAmount`,
    GSTIncluded AS `GSTIncluded`,
    IsCondo,
    Fitout,
    TransactionOriginationTypeID AS `TransactionOriginationTypeID`,
    IsStartingRateConfirmed AS `IsStartingRateConfirmed`,
    ShareLevelID AS `ShareLevelID`,
    IsExpDateConfirmed AS `IsExpDateConfirmed`,
    EscalationType,
    IsTermsConfirmed AS `IsTermsConfirmed`,
    ConfirmedTenantID AS `ConfirmedTenantID`,
    DealingNumber AS `DealingNumber`,
    IsOccupDateConfirmed AS `IsOccupDateConfirmed`,
    RentFreePeriod AS `RentFreePeriod`,
    NoticePeriodMin AS `NoticePeriodMin`,
    NoticePeriodMax AS `NoticePeriodMax`,
    ListingTypeID AS `ListingTypeID`,
    Folio,
    StageID,
    Status,
    ExceptionReason,
    CreatedDate,
    CreatedBy,
    ModifiedDate,
    ModifiedBy,
    LeaseID,
    IsActive,
    BatchID,
    ProviderID,
    HidedBy AS `HidedBy`,
    IsHidden AS `IsHidden`,
    HidedDate AS `HidedDate`,
    HideReasonID AS `HideReasonID`,
    HideReasonComments AS `HideReasonComments`,
    SubHideReasonID AS `SubHideReasonID`,
    MismatchedTaggedBy AS `MismatchedTaggedBy`,
    IsMismatched AS `IsMismatched`,
    Longitude,
    Latitude,
    cast(NULL as string) as Location,
    UID AS `UID`,
    Url AS `Url`,
    AgentName1 AS `AgentName1`,
    AgentEmail1 AS `AgentEmail1`,
    AgentName2 AS `AgentName2`,
    AgentEmail2 AS `AgentEmail2`
FROM
    `arealytics-databricks_unity_catalog`.arealyticscurated.reiv_lease_main
WHERE rn = 1


In [0]:
from pyspark.sql import SparkSession

# Database and JDBC parameters
driver = "org.mariadb.jdbc.Driver"
database_host = "empirical01-uat-ore.cbilzzla5ot8.us-west-2.rds.amazonaws.com"
database_port = "3306"
database_name = "Empirical_DataStage"
table = "LeaseComp_Stage_REIV_For_Grouping"
user = 'dbuser-databricks' 
password = 'DBUserDatabricks#633'

# JDBC URL construction
url = f"jdbc:mysql://{database_host}:{database_port}/{database_name}?useSSL=false"

# Load data from a Spark SQL query
df = spark.sql("""
select *
from
reiv_export
""")

# Write DataFrame to MySQL database, overwriting existing table
df.write \
  .format("jdbc") \
  .mode("append") \
  .option("url", url) \
  .option("dbtable", table) \
  .option("user", user) \
  .option("password", password) \
  .option("driver", driver) \
  .save()

print("Data exported and table overwritten successfully in MySQL database.")
